In [1]:
import pandas as pd
import numpy as np
import math
import helper
import time
import datetime
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.svm import SVC, SVR

from sklearn import metrics
from sklearn.model_selection import KFold, cross_val_score, LeaveOneOut, ShuffleSplit, train_test_split, GridSearchCV 
from sklearn.feature_selection import SelectPercentile, SelectFromModel
import pickle    

# Подгрузка всех таблиц с данными по матчям

In [2]:
# дф с матчами и рейтингом каждой команды с патча 7.01 для Премиум
df_rating_teams_Premium = pd.read_csv('../tabel/table from Datdota/tabel with rating teams PREMIUM (with double Infamous).csv', index_col=0)
# удалить повторяющиеся матчи с Infamous, и сделать реиндекс 
df_rating_teams_Premium = df_rating_teams_Premium.drop(1).reset_index().drop('index', axis=1)
len(df_rating_teams_Premium)

3729

In [3]:
# ДФ с KDA героев для всех
df_KDA_hereos = pd.read_csv('../tabel/table from Datdota/tabel with KDA heroes in match.csv', index_col=0)
len(df_KDA_hereos)

3901

In [4]:
# ДФ c KDA героев для каждой команды индивидуально
df_KDA_hereos_teams = pd.read_csv('../tabel/table from Datdota/tabel with KDA heroes for team in match.csv', index_col=0)
len(df_KDA_hereos_teams)

3777

In [5]:
# ДФ с контрпиками (head-to-head)
df_contrpick = pd.read_csv('../tabel/table from Datdota/tabel elo heroes in match vs enemies (head-to_head, contrPicks).csv', index_col=0)
len(df_contrpick)

3901

In [6]:
# ДФ с типом героев
df_type_heroes = pd.read_csv('../tabel/table from Datdota/tabel summ features heroes PREMIUM (carry, support, nucker...).csv', index_col=0)
len(df_type_heroes)

3901

In [7]:
# ДФ с метовыми героями
df_meta_heroes = pd.read_csv('../tabel/table from Datdota/tabel with AvgElo heroes (meta).csv', index_col=0)
len(df_meta_heroes)
df_meta_heroes.columns

Index(['radiant_H1_AvgElo', 'radiant_H2_AvgElo', 'radiant_H3_AvgElo',
       'radiant_H4_AvgElo', 'radiant_H5_AvgElo', 'dire_H1_AvgElo',
       'dire_H2_AvgElo', 'dire_H3_AvgElo', 'dire_H4_AvgElo', 'dire_H5_AvgElo',
       'match_id'],
      dtype='object')

In [8]:
# ДФ с сигнатурками
df_signatures_heroes = pd.read_csv('../tabel/table from Datdota/tabel with AvgElo heroes for teams (signatures).csv', index_col=0)
len(df_signatures_heroes)
df_meta_heroes.columns

Index(['radiant_H1_AvgElo', 'radiant_H2_AvgElo', 'radiant_H3_AvgElo',
       'radiant_H4_AvgElo', 'radiant_H5_AvgElo', 'dire_H1_AvgElo',
       'dire_H2_AvgElo', 'dire_H3_AvgElo', 'dire_H4_AvgElo', 'dire_H5_AvgElo',
       'match_id'],
      dtype='object')

# Создание основного ДФ для обучения и предсказания

In [9]:
# Соединение всех данных в один ДФ
main = pd.merge(df_rating_teams_Premium, df_KDA_hereos, left_on='match_id', right_on='match_id',how='left')
main = pd.merge(main, df_KDA_hereos_teams, left_on='match_id', right_on='match_id',how='left')
main = pd.merge(main, df_contrpick, left_on='match_id', right_on='match_id',how='left')
main = pd.merge(main, df_type_heroes, left_on='match_id', right_on='match_id',how='left')
main = pd.merge(main, df_meta_heroes, left_on='match_id', right_on='match_id',how='left')
main = pd.merge(main, df_signatures_heroes, left_on='match_id', right_on='match_id',how='left')

# удаление не нужных колонок для обучения
main = main.drop(['match_id', 'start_time', 'radiant_team_id', 'radiant_score', 'dire_team_id',
           'dire_score', 'radiant_name', 'dire_name'], axis=1)
len(main)

3729

In [10]:
main.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3729 entries, 0 to 3728
Data columns (total 365 columns):
radiant_win                           3729 non-null bool
radiant_current_elo32                 3729 non-null float64
radiant_sevenDayAgo_elo32             3722 non-null float64
radiant_sevenDayAvg_elo32             3729 non-null float64
radiant_thirtyDayAgo_elo32            3685 non-null float64
radiant_thirtyDayAvg_elo32            3729 non-null float64
radiant_current_elo64                 3729 non-null float64
radiant_sevenDayAgo_elo64             3722 non-null float64
radiant_sevenDayAvg_elo64             3729 non-null float64
radiant_thirtyDayAgo_elo64            3685 non-null float64
radiant_thirtyDayAvg_elo64            3729 non-null float64
radiant_mu_glicko                     3729 non-null float64
radiant_rating_glicko                 3729 non-null float64
radiant_ratingSevenDaysAgo_glicko     3729 non-null float64
radiant_sigma_glicko                  3729 non-null flo

In [11]:
# Заполнить пустые значения на 0
main = main.fillna(0)


# Создание обучающих и тестовых данных

In [12]:
# удалть имена команд и сделать обучающие данные и ответы
X = main.drop(['radiant_win'], axis=1)
y = main['radiant_win']

# СДЕЛАТЬ 1 или 0 вместо true false
y = y.astype(int)

# # заполнить пропуски на 0
# X = X.fillna(0)

# деление на тестовые и обучающую выборку
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=0)

# Решетчатый поиск на RandomForest

In [13]:
# Параметры RF
param_grid = {'n_estimators': range(10,100, 10), 'max_depth': range(10, 56, 10),
              'max_features': range(40, 365, 40)}
grid_search_RF = GridSearchCV(RandomForestClassifier(), param_grid, cv=5)
grid_search_RF.fit(X_train,y_train)

print("Правильность на тестовом наборе: {:.2f}".format(grid_search_RF.score(X_test, y_test)))
print("Наилучшие значения параметров: {}".format(grid_search_RF.best_params_))
print("Наилучшее значение кросс-валидац. правильности:{:.2f}".grid_search_RF(grid_search.best_score_))

Правильность на тестовом наборе: 0.73
Наилучшие значения параметров: {'max_depth': 40, 'max_features': 200, 'n_estimators': 70}


AttributeError: 'str' object has no attribute 'grid_search_RF'

In [15]:
print("Правильность на тестовом наборе: {:.2f}".format(grid_search_RF.score(X_test, y_test)))
print("Наилучшие значения параметров: {}".format(grid_search_RF.best_params_))
print("Наилучшее значение кросс-валидац. правильности:{:.2f}".format(grid_search_RF.best_score_))

Правильность на тестовом наборе: 0.73
Наилучшие значения параметров: {'max_depth': 40, 'max_features': 200, 'n_estimators': 70}
Наилучшее значение кросс-валидац. правильности:0.75
